# Introduction #



This is the first tutorial of [CeADAR's course in Practical Self-supervised Deep-learning](https://community.ceadar.ie/courses/7086833/content). 

<img src="notebook_images/ssl-logo.jpg" alt="classes" style="width:600px" />

This tutorial will show you how to approach the following tasks.

* Load image data into a torch Tensor object and visualize it.

* Create a custom torch dataset class to load and process a simple image dataset.

* Train a simple convolutional nerual network to solve an image classification task.

* Test the accuracy of your model.

We have written partial code for these tasks and we have set out a series of tasks which require you to complete it.

## Dataset ##

For the purpose of this tutorial we will be using a simple image dataset. This dataset was originally created by Intel for image classification. It originally contained six classes of images but in this tutorial we will just be using three of these classes, buildings, forests and mountains. Examples of these are shown below. 

<img src="notebook_images/class-examples.png" alt="classes" style="width:800px;height:280px;" />


## Loading and Visualizing Image Data ##

When developing models in Pytorch the [Tensor class](https://pytorch.org/docs/stable/tensors.html) provided by torch is used. Tensors are very similar to other data classes like NumPy arrays but differ in a number of important ways. They can be operated on CUDA enabled GPUs and also store gradient information which signifigantly speeds up the calculations carried out when performing back propogation during neural network training.

To read and process the image data we will use the [torchvision library](https://pytorch.org/vision/stable/index.html). This is a popular library which in addition to providing image processing methods also gives easy access to well developed image datasets and pre-trained models.

To visualize the data we can just the [matplotlib library](https://matplotlib.org/).

### Task 1 ###
Using torchvision's [read_image](https://pytorch.org/vision/stable/generated/torchvision.io.read_image.html) function and the Pyplot's [imshow](https://matplotlib.org/3.5.0/api/_as_gen/matplotlib.pyplot.imshow.html) function visualize some image examples. 
#### Hint ####
Torchvision shapes images so that the channel is the first dimension. This is not compatible with imshow you will have to use the [permute method](https://pytorch.org/docs/stable/generated/torch.permute.html) to reorder the tensor dimensions.

In [ ]:
from torchvision.io import read_image
import matplotlib.pyplot as plt


image_path = "./intel_images/train/buildings/4.jpg"

image_tensor = read_image(image_path)

# perform the correct permutation to get the dimensions in order
permuted_tensor = image_tensor.permute((1,2,0))

plt.imshow(permuted_tensor)

## Torch Datasets ##

Torch provides a [dataset class](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) for storing and interacting with training and testing data. Many of torch's mentods are designed to interface directly with torch datasets so it's a good idea to get familiar with them. 

### Task 2 ###

Below is laid out the basic structure of a torch dataset class. Try and complete this section of code to make a custom dataset class for our image dataset.

A torch dataset has three basic methods which must be implemented.

* **init** - This method initializes an instances of this class. In our case it should load the images using the read_images function used previously, and store them in a list. It should also load the target information of the dataset, in this case the class labels of images. These can be stored in several different ways eg. strings, integers etc. but in this you should store them as [one hot encodings](https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/) ie. tensors of shape (3). This will make training our model simpler.

* **getitem** - This method is how we access the data contained in the dataset given an index integer. This method should return a pair consisting of an image tensor and its accompanying class label tensor. 

* **len** - This method simply returns the number of examples in the dataset.

#### Hint ####

When reading in the images it's important that they are all the same size. You can use the [transforms feature](https://pytorch.org/vision/stable/transforms.html) in torchvision to resize the images to a standard shape. For the purposes of this exercise I would reccomend 16x16, but feel free to try different sizes. A larger image size might lead to better results, but will mean that your model will take longer to train. 

In [ ]:
import torch
from torch.utils.data import Dataset
import os
import torchvision.transforms as T

class Intel(Dataset):
    
    # The init function given a path to the dataset folder loads in the data and stores it. In this case 
    # you will have to cycle through the folders containing different classes of images, using the 
    # read_image fucntion to load the data into tensors. The class labels should also be stored in the form of a 
    # 3 dimensional one hot encoding vector. 
    # We have also included an option to load the training or test data provided.
    
    def __init__(self, dataset_path="intel_images", train=True):
        
        # store image tensors here
        self.images = []
        
        # store target vectors here
        self.targets = []
        
        if train:
            images_path = dataset_path + "/train"
        else:
            images_path = dataset_path + "/test"
        class_index = 0
        image_classes = [f.path for f in os.scandir(images_path) if f.is_dir()]
        for class_path in image_classes:
            image_paths = [f.path for f in os.scandir(class_path)]
            for image in image_paths:
                img = read_image(image)/255
                self.images.append(img)
                target_vector = torch.zeros(3)
                target_vector[class_index] = 1.0
                self.targets.append(target_vector)
            class_index += 1
    
    # This is the method used to access the data in the dataset class object. Given an index integer it should 
    # return a tuple containing and image tensor and it's accompyaning class vector.
    
    def __getitem__(self, index):
        transform = T.Resize((16,16))
        img = self.images[index]
        target = self.targets[index]
        return transform(img), target
    
    # This should return the number of examples in the dataset
    
    def __len__(self):
        return len(self.targets)

        
        

After writing the dataset class we can declare two instances of it, one for training our model and one for testing it.

In [ ]:

train_data = Intel(train=True)
test_data = Intel(train=False)


When developing a model using Pytorch we generally use [Dataloaders](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html). These allow us to easily load batches from a torch dataset, as well as automatically shuffling the data.

In [ ]:
from torch.utils.data import DataLoader


train_loader = DataLoader(train_data, batch_size=32,
                                          shuffle=True)
test_loader =  DataLoader(test_data, batch_size=32,
                                          shuffle=True)

## Designing a Model ##

When dealing with visual data such as images or videos [convolutional layers](https://www.ibm.com/cloud/learn/convolutional-neural-networks) are generally made use of. These kinds of layers are very good at learning visual features such as edges, color gradients and shapes. Convolutional networks for classification generally consist of several convolutional layers for feature extraction, followed by one or more fully connected layers which perform the classification. This can be seen in the diagream below.

<img src="notebook_images/cnn-architecture.png" alt="network" width="500"/>

### Task 3 ###

Below is the code for a very simple convolutional network containing two convolutional layers and two fully connected layers. The arguments for each layer have been removed. Try and fill them in. Keep in mind the size of the input and the fact that the output should be tensor of shape (3) in order to match the target vectors in the dataset.

#### Hint ####

Make sure the layer sizes are matched up with eachother. In order for the output of a convolutional layer to be successfully input into a fully connected linear layer it must be flattened. It may take some trial and error to determine the size of this flatten vector, and hene how big the fully connected should be. Try some examples from the dataset and see what length of vector this flattening produces.

In [ ]:
from torch import nn
import torch.nn.functional as F

class model(nn.Module):
    
    # When initializing a model instance we declare what layers the model contains. 
    # In this case we have 2 conv2d layers and two fully connected linear layers.
    # The final layer has size 3 as this is the number of classes we are dealing with.
    
    def __init__(self):
        super(model, self).__init__()

        self.conv1 = nn.Conv2d(3,32,(3,3))
        self.conv2 = nn.Conv2d(32,16,(3,3))
        self.fc1 = nn.Linear(2304,100)
        self.final = nn.Linear(100,3)
    
    # The forward method defines how an input is processed in the model.
    # In this case the input is passed through the convolutional layers 
    # and then flattened so that it can be proessed by the linear layers.
    # At the end we apply a softmax function. This is standard for classification
    # models.
    def forward(self, input1):
        
        out = self.conv1(input1)
        out = self.conv2(out)
        out = torch.flatten(out,1)
        out = F.relu(self.fc1(out))
        out = self.final(out)
        out = F.softmax(out,dim=1)
        return out
    
test_model = model()

Use the code below to make sure your model's layer sizes are configured correctly. The output should be a tensor of shape (3).

In [ ]:
test_input = train_data[0][0].reshape((1,3,16,16))
test_output = test_model(test_input)

print(test_output)

## Model Training ##

Now that we have a dataset and a model we can get to training. When training there are two important objects, the loss function and the optimizer.

#### Loss Function ####

The loss function of a neural network is the function which is being optimized during training, either being maximized or minimized. This is done by updating the neural network parameters, known as weights. In this case we are using torch's cross entropy loss function. This loss function basically measures the difference between our model's predictions and the true class values. Obviously we want to minimize this as much as possible. 

There are many different popular loss functions for classification and regression problems. Much of the research carried out in deep learning consists in developing new loss functions for novel tasks. This [article](https://machinelearningmastery.com/loss-and-loss-functions-for-training-deep-learning-neural-networks/) gives a good overview of popular loss functions.

#### Optimizer ####

The optimizer is the algorithm which actually updates the network weights during training in order to minimize the loss function. In this case we use one of the most popular optimizers, stochastic gradient descent. This is similar to a normal gradient descent method used during traditional machine learning, except in this case the optimizer updates the model weights for every batch, rather than for the whole training set at once.

There are a wide variety of optimizers, each of which is better suited to some tasks than others. This [article](https://medium.com/mlearning-ai/optimizers-in-deep-learning-7bf81fed78a0) provides a good summary of other popular optimization algorithms.

### Task 4 ###

The most important optimizer parameter is the learning rate. This determines how much the network weights are updated druing every optimization. If it is too high then the optimizer might overshoot the loss function's minimum, if it is too low then the model could take too long to train. This is visualized is the diagram below.

<img src="notebook_images/learning_rate.png" alt="network" width="600"/>

You should try to experiment with different learning rates to see what effect it has on model training. Try starting at 0.001 and moving up to 1. 

In [ ]:
import torch.optim as optim

# choose the loss function
criterion = nn.CrossEntropyLoss()

# declare which parameters are being optimized by the optimizer. 
# in this case it is just all the network parameters
target_params = test_model.parameters()

# declare the optimizer object and set the learning rate
optimizer = optim.SGD(target_params, lr=0.003)

### Training Loop ###

Below is a standard loop for model training. Unlike libraries such as [Keras](https://keras.io/), which implement training procedures for you, Pytorch requires you to explicitly write your own training loop. These might look complicated but they all follow the same structure. The most important section of the loop consists of the three parts explained below.

* **Forward** - The forward pass is when the batch of our training examples is feed forward through the network in orderr to produce predictions.
* **Backward** - Here the back propogation algorithm is used to calculate the gradient of every parameter in the network with respect to the batch loss function. The loss is calculated from the model predictions and the true values.
* **Optimize** - Using the gradients calculated in the previous step the optimizer updates the network weights in order to reduce to loss value. 

It's definitely not neccessary to understand all the mathematical details of the above steps, but it is good to have a general idea of what is happening in the training loop. 



In [ ]:
from tqdm import tqdm


for epoch in range(10):  # loop over the dataset multiple times
    print('Epoch:'+str(epoch+1))
    
    epoch_loss = 0
    
    with tqdm(train_loader, unit="batch") as tepoch:
        running_loss = 0.0
        for i, data in enumerate(tepoch, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            
            # zero the parameter gradients
            
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = test_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
    
    print("epoch loss was: " + str(epoch_loss))

## Model Testing ##

There are many different metrics which can be used to measure the performane of a model inclduing basic accuracy, precision, recall etc. For the purposes of this tutorial we will just use simple accuracy.

### Task 5 ###

Write a function which takes a model as input and returns its accuracy on the test dataset loaded earlier. This is most easily done by simply looping over the testset examples and determing whether the model prediction is correct.

#### Hint ####

The true target values are one hot encoding vectors ie. (1,0,0), (0,1,0) etc. The model outputs are not likely to look like this so you will have to determine which class has the highest score in the model's predictions. For example the model could output the vector (0.7,0.2,0.1). For the purposes of calculating accuracy this should be treated as (1,0,0).

In [ ]:
def test_accuracy(model):
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            inputs, labels = data
            total += labels.size(0)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            _, labels = torch.max(labels.data, 1)
            correct += (predicted == labels).sum().item()
    return correct/total

In [ ]:
model_test_accuracy = test_accuracy(test_model)

print("Model accuracy on test set is " + str(model_test_accuracy*100) + "%")

## Conclusion ##

In this tutorial we have shown you how to write a basic dataset class to load and process a simple image classification dataset. We also showed you how to construct a simple a model and then train and test it. Fell free to experiment further by trying different model configurations or training with different batch sizes.